# Lost Luggage Distribution  Problem

## Objective and Prerequisites

This model is an example of a vehicle routing problem. A small company with six vans has a contract with a number of airlines to pick up lost or delayed baggage and deliver each piece of luggage to its owner. This problem is formulated as a binary optimization problem using the Gurobi Python API and solved with the Gurobi Optimizer.

This model is example 27 from the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 287-289 and 343-344.

This modeling example is at the advanced level, where we assume that you know Python and the Gurobi Python API and that you have advanced knowledge of building mathematical optimization models. Typically, the objective function and/or constraints of these examples are complex or require advanced features of the Gurobi Python API.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 

**Gurobi License** <br /> 
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an [evaluation license](https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-MUI-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_Lost_Luggage_Distribution_COM_EVAL_GitHub&utm_term=Lost%20Luggage%20Distribution&utm_content=C_JPM) as a *commercial user*, or download a [free license](https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-EDU-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_Lost_Luggage_Distribution_COM_EVAL_GitHub&utm_term=Lost%20Luggage%20Distribution&utm_content=C_JPM) as an *academic user*.

## Problem Description

A small company with six vans has a contract with a number of airlines to pick up lost or delayed baggage, belonging to customers in the London area, from Heathrow airport at 6 p.m. each evening. The contract stipulates that each customer must have their baggage delivered by 8 p.m. The company requires a model to advise them what is the minimum number of vans they need to use and to which customers each van should deliver and in what order. There is no practical capacity limitation on each van. Each van can hold all baggage that needs to be delivered in a two-hour period. To solve this problem, we can formulate an optimization model that minimizes the number of vans that need to be used.



##  Model Formulation


### Sets and Indices

$i,j \in \text{Locations} \equiv L=\{0,1..(n-1)\}$: Set of locations where $0$ is the index for the single depot -Heawthrow airport, and $n$ is the number of locations.

$k \in \text{Vans} \equiv  V=\{0..K-1\}$: Index and set of vans, where $K$ is the number of vans.

$S_k \in S  $: Tour of van $k$, i.e. subset of locations visited by the van.

### Parameters

$t_{i,j} \in \mathbb{R}^+$: Travel time from location $i$  to location $j$.

### Decision Variables

$x_{i,j,k} \in \{0,1 \}$: This binary variable is equal 1, if van $k$ visits and goes directly from location $i$ to location $j$, and zero otherwise.

$y_{i,k} \in \{0,1 \}$: This binary variable is equal 1, if van $k$ visits location $i$, and zero otherwise.

$z_{k} \in \{0,1 \}$: This binary variable is equal 1, if van $k \in \{1,2..K\}$ is used, and zero otherwise.

### Objective Function

**Number of vans**: Minimize number of vans used.

\begin{equation}
\text{Minimize} \quad \sum_{k = 1}^{K} z_k
\end{equation}

### Constraints

**Van utilization**: For all locations different from the depot, i.e. $i > 0$, if the location is visited by van $k$, then it is used.

\begin{equation}
y_{i,k} \leq z_{k} \quad \forall i \in L \setminus \{0\}, \; k \in V
\end{equation}

**Travel time**: No van travels for more than 120 min. Observe that the travel time to return to the depot is not considered.

\begin{equation}
\sum_{i \in L} \sum_{j \in L \setminus \{0\}} t_{i,j}*x_{i,j,k} \leq 120*z_k \quad \forall k \in  V
\end{equation}

**Visit all customers**:  Each customer location is visited by exactly one van.

\begin{equation}
\sum_{k \in V}  y_{i,k} = 1 \quad \forall i \in L \setminus \{0\}
\end{equation}

**Depot**: Heathrow is visited by every van used. 

\begin{equation}
\sum_{k \in V}  y_{1,k} \geq \sum_{k \in V} z_k
\end{equation}

**Arriving at a location**: If location $j$ is visited by van $k$, then the van is coming from another location $i$.

\begin{equation}
\sum_{i \in L}  x_{i,j,k} =  y_{j,k}  \quad \forall j \in L, \; k \in V
\end{equation}

**Leaving a location**: If van $k$ leaves location $j$, then the van is going to another location $i$.

\begin{equation}
\sum_{i \in L}  x_{j,i,k} = y_{j,k}  \quad \forall j \in L, \; k \in V
\end{equation}

**Breaking symmetry**: 

\begin{equation}
\sum_{i \in L}  y_{i,k} \geq \sum_{i \in L}  y_{i,k+1} \quad \forall k \in  \{0..K-1\}
\end{equation}

**Subtour elimination**: These constraints ensure that for each van route, there is no cycle. 

\begin{equation}
\sum_{(i,j) \in S_k}x_{i,j,k} \leq |S_k|-1 \quad \forall  k \in K, \;   S_k \subseteq L
\end{equation}



## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [1]:
import sys
import math
import random
from itertools import permutations
import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.1.0

### Callback Definition
The following function determines lazy constraints to eliminate sub-tours.

In [2]:
# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._x)
        for k in vans:
            for i,j in time.keys():
                if vals[i, j, k] > 0.5:
                    selected = gp.tuplelist((i, j) for i, j, k in model._x.keys()
                                if vals[i, j, k] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n: 
            for k in vans:
                model.cbLazy(gp.quicksum(model._x[i, j, k]
                                         for i, j in permutations(tour, 2))
                             <= len(tour)-1)


# Given a tuplelist of edges, find the shortest subtour not containing depot
def subtour(edges):
    unvisited = list(range(1, n))
    cycle = range(n+1)  # initial length has 1 more city
    # First, remove all nodes connected to depot
    depot_connected = [j for i, j in edges.select(0, '*')]
    while depot_connected:
        current = depot_connected.pop()
        unvisited.remove(current)
        neighbors = [j for i, j in edges.select(current, '*')
                     if j in unvisited and j != 0]
        depot_connected += neighbors

    # Now, find subtour using tsp.py code
    while unvisited:
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

## Input data  
We define all the input data for the model. The user defines the number of locations, including the depot, and the number of vans. We randomly determine the coordinates of each location and then calculate the Euclidean distance between each pair of locations. We assume a speed of 60 km/hr, which is 1 km/min. Hence travel time is equal to the distance.

In [3]:
# number of locations, including the depot. The index of the depot is 0
n = 17
locations = [*range(n)]

# number of vans
K = 6
vans = [*range(K)]

# Create n random points
# Depot is located at (0,0) coordinates
random.seed(1)
points = [(0, 0)]
points += [(random.randint(0, 50), random.randint(0, 50)) for i in range(n-1)]

# Dictionary of Euclidean distance between each pair of points
# Assume a speed of 60 km/hr, which is 1 km/min. Hence travel time = distance
time = {(i, j):
        math.sqrt(sum((points[i][k]-points[j][k])**2 for k in range(2)))
        for i in range(n) for j in range(n) if i != j}

# Preprocessing
l_ijk = []
for i,j in time.keys():
    for k in range(K):
        tp = i,j,k
        l_ijk.append(tp)
        
ijk = gp.tuplelist(l_ijk)
#print(ijk)

l_ik = []
for i in locations:
    for k in vans:
        tp = i,k
        l_ik.append(tp)
        
ik = gp.tuplelist(l_ik)
#print(ik)

## Model Deployment

We create a model and the variables. The decision variables determines the order in which each van visits a subset of custormers, which customer is visited by each van, and if a van is used or not.

In [4]:
m = gp.Model('lost_luggage_distribution.lp')

# Create variables: 

# x =1, if van  k  visits and goes directly from location  i  to location  j 
x = m.addVars(ijk, vtype=GRB.BINARY, name='FromToBy')

# y = 1, if customer i is visited by van k
y = m.addVars(ik, vtype=GRB.BINARY, name='visitBy')

# Number of vans used is a decision variable
z = m.addVars(vans, vtype=GRB.BINARY, name='used')

Using license file c:\gurobi\gurobi.lic


## Constraints

For all locations different from depot, i.e. $i > 0$, if the location is visited by van $k$, then it is used.

In [5]:
# Van utilization constraint

visitCustomer = m.addConstrs((y[i,k] <= z[k]  for k in vans for i in locations if i > 0), name='visitCustomer' )

No van travels for more than 120 min.

In [6]:
# Travel time constraint
# Exclude the time to return to the depot

travelTime = m.addConstrs((gp.quicksum(time[i,j]*x[i,j,k] for i,j in time.keys() if j > 0) <= 120*z[k] for k in vans), 
                          name='travelTime' )

Each customer location is visited by exactly one van

In [7]:
# Visit all customers
visitAll = m.addConstrs((gp.quicksum(y[i,k] for k in vans ) == 1 for i in locations if i > 0), 
                          name='visitAll' )

Heathrow (depot) is visited by every van used.

In [8]:
# Depot constraint
depotConstr = m.addConstr((gp.quicksum(y[0,k] for k in vans ) >= z.sum() ), name='depotConstr' )

If location  j  is visited by van  k , then the van is coming from another location  i.

In [9]:
# Arriving at a customer location constraint
ArriveConstr = m.addConstrs((gp.quicksum(x[i,j,k] for i,jj,kk in ijk if (jj == j and kk == k) ) 
                             == y[j,k] for j,k in ik  ), name='ArriveConstr' )

 If van  k  leaves location  j , then the van is going to another location  i.

In [10]:
# Leaving a customer location constraint
LeaveConstr = m.addConstrs((gp.quicksum(x[j,i,k] for jj,i,kk in ijk if (jj == j and kk == k) ) 
                             == y[j,k] for j,k in ik  ), name='LeaveConstr' )

Breaking symmetry constraints.

In [11]:
breakSymm = m.addConstrs((gp.quicksum(y[i,k] for i in locations ) >= 
                          gp.quicksum(y[i,k+1] for i in locations  ) for k in range(K-1) ), name='breakSymm' )

### Objective Function
Minimize number of vans used.

In [12]:
m.setObjective((z.sum() ), GRB.MINIMIZE)

In [13]:
# Verify model formulation

m.write('lost_luggage_distribution.lp')

# Run optimization engine`
m._x = x
m.Params.LazyConstraints = 1
m.optimize(subtourelim)

Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 328 rows, 1740 columns and 5480 nonzeros
Model fingerprint: 0xe25b160d
Variable types: 0 continuous, 1740 integer (1740 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.02s
Presolved: 328 rows, 1740 columns, 5480 nonzeros
Variable types: 0 continuous, 1740 integer (1740 binary)

Root relaxation: objective 1.492625e+00, 908 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.49263    0   54          -    1.49263      -     -    0s
     0     0    1.55594    0  181          -   

## Analysis

The optimal route of each van used and the total lost luggage delivery time report follows.

In [14]:
# Print optimal routes
vals = m.getAttr('X', x)

# Travel to depot is zero
ttime = {}
for i,j in time.keys():
    ttime[i,j] = time[i,j]
    if j == 0:
        ttime[i,j] = 0

for k in vans:
    selected = gp.tuplelist((i, j) for i, j, kk in vals.keys() if (kk == k and vals[i, j, kk] > 0.5) )
    for i, tup in enumerate(selected.select(0, '*')):
        travelTime = 0
        print(f"Route for van {k}: 0", end='')
        travelTime += ttime[tup[0],tup[1]]
        neighbor = tup[1]
        while neighbor:
            print(f" -> {neighbor}", end='')
            current = neighbor
            neighbor = selected.select(neighbor, '*')[0][1]
            travelTime += ttime[current,neighbor]
        print(f" -> 0. Travel time: {round(travelTime,2)} min")

Route for van 0: 0 -> 9 -> 8 -> 12 -> 1 -> 5 -> 6 -> 14 -> 7 -> 2 -> 0. Travel time: 117.98 min
Route for van 1: 0 -> 3 -> 16 -> 15 -> 13 -> 10 -> 11 -> 4 -> 0. Travel time: 118.37 min


## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright © 2020 Gurobi Optimization, LLC